In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import os
import creep_event_picker as cep
import h5py
import re
import matplotlib.dates as mdates
%matplotlib qt


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
def check_dir(path):
    isExist = os.path.exists(path)
    if not isExist:
        # Create a new directory because it does not exist 
        os.makedirs(path, exist_ok=True)

Import creepmeter database

In [3]:
'''Creepmeter_dataframe = pd.read_excel('../../Data/Creepmeter_list.xlsx')
#Creepmeter_dataframe.drop(Creepmeter_dataframe[Creepmeter_dataframe['Creepmeter_abbrv']!='NS45'].index,inplace=True)'''

Creepmeter_dataframe = pd.read_csv('../../Data/DATA_tidied/creepmeter_metadata_post_standardisation_sac_codes_updated.csv',index_col=0)
Creepmeter_dataframe.drop_duplicates('Creepmeter_abbrv',inplace=True)
Creepmeter_dataframe.drop(Creepmeter_dataframe[Creepmeter_dataframe['Creepmeter_abbrv']=='XMBC'].index,inplace=True)
Creepmeter_dataframe.drop(Creepmeter_dataframe[Creepmeter_dataframe['Creepmeter_abbrv']=='TABC'].index,inplace=True)
Creepmeter_dataframe.reset_index(inplace=True,drop=True)

In [4]:
Creepmeter_dataframe['Creepmeter_abbrv'].unique()

array(['BAL1', 'C461', 'C462', 'CAN1', 'CER1', 'CFW1', 'CHA1', 'CHE1',
       'CHP1', 'COZ1', 'CPP1', 'CRR1', 'CTM1', 'CWC3', 'CWN1', 'DS30',
       'DU30', 'ESZ1', 'FCR1', 'FE30', 'GB', 'GD', 'GOK1', 'GOZ1', 'GS',
       'HAM1', 'HAS1', 'HAT1', 'INW1', 'IR30', 'ISW1', 'KAR1', 'KIR1',
       'NS45', 'ORM1', 'PAN1', 'PAS1', 'PET1', 'QE30', 'QW30', 'R7', 'RA',
       'RB', 'RR45', 'SC30', 'SEL1', 'SH30', 'SIV1', 'SJN1', 'SU30',
       'TAS1', 'TEP1', 'WKR1', 'WSZ1', 'X461', 'XGH1', 'XHR1', 'XHR2',
       'XHR3', 'XHSW', 'XMD1', 'XMM1', 'XMR1', 'XPK1', 'XPK2', 'XRSW',
       'XSC1', 'XSH1', 'XSJ2', 'XSJ3', 'XTA1', 'XVA1', 'YZE1', 'YZW1',
       'COML', 'COLE', 'COLW', 'EASH'], dtype=object)

In [5]:
frequency_dict = {
        1/60: '1S',    # 1 second
        1/6: '10S',  # 10 seconds
        1/2: '30S',  # 30 seconds
        1: '1T',  # 1 minute
        2: '2T',  # 2 minutes
        5: '5T',  # 5 minutes
        10: '10T',# 10 minutes
        15: '15T', # 15 minutes
        30: '30T',# 30 minutes
        60: '60T'    # 60 minutes
    }

In [7]:
plt.close('all')
for i in range(len(Creepmeter_dataframe)):
    with h5py.File('../../Data/DATA_tidied/HDF5/{k}.h5'.format(k=Creepmeter_dataframe['Creepmeter_abbrv'].iloc[i]), 'r') as f:
        # Loop through each key in the file
        keys = list(f.keys())
        for key in f.keys():
            if key not in ['Temperature','Temperature_1T','Temperature_5T','Daily_measurements','Manual_measurements','Orthogonal']:
                print("Key:", key)
                #print('extracting data')
                data = f[key]  # Access the dataset
                data_keys = list(data.keys())
                #print(data_keys)
                slip = data[data_keys[0]][:]
                time =  data[data_keys[1]][:]
                decoded_time = [byte_str.decode('utf-8') for byte_str in time]
                decoded_time = pd.to_datetime(decoded_time)





                match = re.search(r'_(\d*\.?\d+)mins$', key)
                if match:
                    freq_value = float(match.group(1))
                    new_suffix = frequency_dict.get(freq_value)
                    if new_suffix:
                        new_key = re.sub(r'_\d*\.?\d+mins$', f'_{new_suffix}', key)
                    else:
                        new_key = key  # or raise an error / warning
                else:
                    new_key = key  # unchanged if pattern not matched

                #print(new_key)
                try:
                    df_picks = pd.read_csv("../../Data/DATA_tidied/Picks/{q}_picks.csv".format(q =new_key),index_col=0)
                except FileNotFoundError:
                    print('No events')
                    continue

                
                QC = []
                df_end_final=pd.DataFrame(columns=['ET'])
                df_start_final=pd.DataFrame(columns=['ST'])
                done = os.path.isfile("../../Data/DATA_tidied/Picks/Final/{q}_picks_final.csv".format(q =new_key))
                if done == True:
                    print("Alread Qc'ed")
                else:
                    print('Identifying events')
                    df_picks.drop(df_picks[df_picks['Quality_check_new'] != 'CE'].index,inplace=True)
                    df_picks.sort_values(by='ST_new',inplace=True)
                    df_picks.reset_index(inplace=True, drop=True)
                    if len(df_picks) == 0:
                        continue

                    print('beginning QC for {k}'.format(k=new_key))
                    
                    for p in range(len(df_picks)):

                        fig = plt.figure(figsize=(15,5))
                        ax = plt.subplot(1,1,1)

                        fig.suptitle('events {a} of {c}'.format(a = p+1,c =len(df_picks) ))
                        boolarr_selecta = np.logical_and(np.array(decoded_time)>=pd.to_datetime(df_picks['ST_new'].iloc[p])-dt.timedelta(hours=72)
                                                            ,np.array(decoded_time)<=pd.to_datetime(df_picks['ET_new'].iloc[p])+dt.timedelta(hours = 72))
                        tm_selecta = decoded_time[boolarr_selecta]


                        creep_selecta = slip[boolarr_selecta]
                        creep_selecta = creep_selecta - creep_selecta[0]

                        bool_event = np.logical_and(np.array(decoded_time)>=pd.to_datetime(df_picks['ST_new'].iloc[p])
                                                            ,np.array(decoded_time)<=pd.to_datetime(df_picks['ET_new'].iloc[p]))
                        
                        creep_event = slip[bool_event]
                        disp = creep_event[-1] - creep_event[0]

                        if disp >=0.02:
                            color = 'green'
                        else:
                            color = 'red'

                        if 'Quality_check_final' in df_picks.columns:
                            if df_picks['Quality_check_final'].iloc[p]=='CE':
                                marker = 'o'
                            else:
                                marker='s'
                        else:
                            marker = 'o'

                        plt.plot(tm_selecta,creep_selecta,color=color,marker=marker,markersize=6,markerfacecolor='none',)
                        left,right = ax.get_xlim()
                        top,bottom = ax.get_ylim()
                        plt.ylim(top,bottom)
                        plt.xlim(left,right)
                        

                        plt.vlines([df_picks['ST_new'].iloc[p],df_picks['ET_new'].iloc[p]],ymin=bottom,ymax=top,colors=['red','green'])
                        plt.hlines([0.02],xmin=left,xmax=right,colors=['orange'])
                        plt.ylabel('mm')
                    
                        scale = 1.1
                        zp = cep.ZoomPan()
                        figZoom = zp.zoom_factory(ax, base_scale = scale)
                        figPan = zp.pan_factory(ax)
                        s = 1
                        fig.set_size_inches(11,11)
                        fig.tight_layout()
                        plt.show()
                        ##### QC ####
                        print('{m}/{n}'.format(m=p+1,n=len(df_picks)))  
                        print('is the start pick good?\n esc=yes, click=no')
                        check_start = plt.ginput(1,timeout = -1)
                        if len(check_start) ==1:
                            for sublist in check_start:
                                df = pd.DataFrame({'ST':[sublist[0]]})
                                df_start_final = pd.concat([df_start_final,df])
                            #print('new start picked\n {k}'.format(k=mdates.num2date(sublist[0])))
                        else:
                            print('start all good! :D')
                            try:
                                df = pd.DataFrame({'ST':mdates.date2num([df_picks.ST_new.iloc[p]])})
                                print('start picked\n {k}'.format(k=mdates.num2date(df.ST_new.iloc[0])))
                            except:
                                df = pd.DataFrame({'ST':mdates.date2num([df_picks.ST.iloc[p]])})
                                print('start picked\n {k}'.format(k=mdates.num2date(df.ST.iloc[0])))
                            df_start_final = pd.concat([df_start_final,df])
                            
                        print("now pick the end\n esc=keep current, click=pick new")
                        pts = plt.ginput(1,timeout = -1)
                        if len(pts)==1:
                            for sublist in pts:
                                df = pd.DataFrame({'ET':[sublist[0]]})
                                df_end_final = pd.concat([df_end_final,df])
                                #print('new end picked\n {k}'.format(k=mdates.num2date(sublist[0])))
                        else:
                            print('end all good! :D')
                            try:
                                df = pd.DataFrame({'ET':mdates.date2num([df_picks.ET_new.iloc[p]])})
                                print('end picked\n {k}'.format(k=mdates.num2date(df.ET_new.iloc[0])))
                            except:
                                df = pd.DataFrame({'ET':mdates.date2num([df_picks.ET.iloc[p]])})
                                print('end picked\n {k}'.format(k=mdates.num2date(df.ET.iloc[0])))
                            df_end_final = pd.concat([df_end_final,df])
                            
                        
                        
                        print('is this a creep event?\n esc=no, click=yes')
                        check_pts = plt.ginput(1,timeout = -1)
                        
                        if len(check_pts) == 1:
                            QC.append('CE')
                            print('YES!! :D')
                        else:
                            QC.append('NCE')
                            print('Nope :(')

                        plt.close()
                        ends_list=[]
                        for r in range(len(df_end_final)):
                            time_convert = mdates.num2date(float(df_end_final.ET.iloc[r]))
                            nearest_e = decoded_time[np.argmin(np.abs(np.array(decoded_time) - np.datetime64(time_convert)))]
                            ends_list.append(nearest_e)
                        start_list=[]
                        for r in range(len(df_end_final)):
                            time_convert = mdates.num2date(float(df_start_final.ST.iloc[r]))
                            nearest_s = decoded_time[np.argmin(np.abs(np.array(decoded_time) - np.datetime64(time_convert)))]
                            start_list.append(nearest_s)
                    
                    
                    df_picks['ST_final'] = start_list
                    df_picks['ET_final'] = ends_list
                    df_picks['Quality_check_final'] = QC
                    df_picks.to_csv("../../Data/DATA_tidied/Picks/Final/{q}_picks_final.csv".format(q =new_key))
                    
                    print(df_picks)
                    print('QC completed')

Key: BAL1_1.0mins
Identifying events
beginning QC for BAL1_1T
1/10
is the start pick good?
 esc=yes, click=no
start all good! :D
start picked
 2023-03-12 10:47:00+00:00
now pick the end
 esc=keep current, click=pick new


/var/folders/56/47k8q3q1449cq8kwzkz5r8ch0000gr/T/ipykernel_6854/3037488672.py:126: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_start_final = pd.concat([df_start_final,df])


end all good! :D
end picked
 2023-03-13 18:56:00+00:00
is this a creep event?
 esc=no, click=yes


/var/folders/56/47k8q3q1449cq8kwzkz5r8ch0000gr/T/ipykernel_6854/3037488672.py:143: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_end_final = pd.concat([df_end_final,df])


Nope :(
2/10
is the start pick good?
 esc=yes, click=no


/var/folders/56/47k8q3q1449cq8kwzkz5r8ch0000gr/T/ipykernel_6854/3037488672.py:161: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  nearest_e = decoded_time[np.argmin(np.abs(np.array(decoded_time) - np.datetime64(time_convert)))]
/var/folders/56/47k8q3q1449cq8kwzkz5r8ch0000gr/T/ipykernel_6854/3037488672.py:166: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  nearest_s = decoded_time[np.argmin(np.abs(np.array(decoded_time) - np.datetime64(time_convert)))]


start all good! :D
start picked
 2023-03-14 09:22:00+00:00
now pick the end
 esc=keep current, click=pick new
is this a creep event?
 esc=no, click=yes
YES!! :D
3/10
is the start pick good?
 esc=yes, click=no
now pick the end
 esc=keep current, click=pick new
end all good! :D
end picked
 2023-03-21 08:26:00+00:00
is this a creep event?
 esc=no, click=yes
YES!! :D
4/10
is the start pick good?
 esc=yes, click=no
start all good! :D
start picked
 2023-08-22 15:22:00+00:00
now pick the end
 esc=keep current, click=pick new
end all good! :D
end picked
 2023-08-23 03:53:00+00:00
is this a creep event?
 esc=no, click=yes
YES!! :D
5/10
is the start pick good?
 esc=yes, click=no
start all good! :D
start picked
 2023-08-23 03:53:00+00:00
now pick the end
 esc=keep current, click=pick new
end all good! :D
end picked
 2023-08-23 07:30:00+00:00
is this a creep event?
 esc=no, click=yes
YES!! :D
6/10
is the start pick good?
 esc=yes, click=no
start all good! :D
start picked
 2023-08-23 07:30:00+00:00

KeyboardInterrupt: 

In [ ]:
df_picks['ST_new'].iloc[p]

In [ ]:
df_picks['ET_new'].iloc[p]

In [ ]:
for i in range(len(Creepmeter_dataframe)):
    with h5py.File('../../Data/DATA_tidied/HDF5/{k}.h5'.format(k=Creepmeter_dataframe['Creepmeter_abbrv'].iloc[i]), 'r') as f:
        # Loop through each key in the file
        keys = list(f.keys())
        for key in f.keys():
            if key not in ['Temperature','Temperature_1T','Temperature_5T','Daily_measurements','Manual_measurements','Orthogonal']:
                #print("Key:", key)
                match = re.search(r'_(\d*\.?\d+)mins$', key)
                if match:
                    freq_value = float(match.group(1))
                    new_suffix = frequency_dict.get(freq_value)
                    if new_suffix:
                        new_key = re.sub(r'_\d*\.?\d+mins$', f'_{new_suffix}', key)
                    else:
                        new_key = key  # or raise an error / warning
                else:
                    new_key = key  # unchanged if pattern not matched

                try:
                    final = '../../Data/DATA_tidied/Picks/Final/{p}_picks_final.csv'.format(p=new_key)
                    final_picks = pd.read_csv(final)
                except FileNotFoundError:
                    dummy=1
                    #print('skipping',f'{new_key}')
                    continue
                

                older = '../../Data/DATA_tidied/Picks/{p}_picks.csv'.format(p=new_key)
                older_picks = pd.read_csv(older)

                if 'Quality_check_final' in older_picks.columns:
                    print(new_key)
